# 13. Volatility and Interest Rate Options

## Overview

* The most liquid nonlinear OCT products are SOFR caps, floors and SOFR swaptions
* This chapter is an overview of SOFR caps, floors and SOFR swaptions, and their pricing and heding

## Motivation

* In rates options business, there are exotic products of 

* More complicated models are calibrated to caps, floors and swaptions quotes in order to price or hedge more exotic products
* Exotic option desk usually trade with the vanilla desk (which  )